In [1]:
# Convex Optimization
# Tutorial 12
# Robust LP

import numpy as np
import cvxpy as cp

In [2]:
np.random.seed(10)
(m, n) = (30, 10)
A = np.random.rand(m, n)
A = np.asmatrix(A)
B = np.random.rand(m,1)
B = np.asmatrix(B)
C_nom = np.ones((n, 1)) + np.random.rand(n, 1)
C_nom = np.asmatrix(C_nom)

F = np.zeros((22, 10))
for i in range(n):
  F[i][i] = 1
  F[n + i][i] = -1
  F[2 * n][i] = 1 / n
  F[2 * n+1][i] = -1 / n

g = np.zeros((22, 1))
for i in range(n):
    g[i] = 1.25 * C_nom[i]
    g[n + i] = -0.75 * C_nom[i]
    g[2 * n] = np.reshape(np.sum(C_nom), (1, 1))/n
    g[2 * n+1] = np.reshape(np.sum(C_nom), (1, 1))/n


In [3]:
# robust LP problem with polyhedral cost uncertainity 
X = cp.Variable((n, 1))
L = cp.Variable((2 * n+2,1))
Objective = cp.Minimize(L.T * g)
Constraints = [A * X >= B, L >= 0, F.T * L == X]
prob = cp.Problem(Objective, Constraints)
prob.solve()
print(f'Optimal cost from robust LP problem is {float(C_nom.T@X.value)}')

C = cp.Variable((n, 1))
Objective = cp.Maximize(C.T * X.value)
Constraints = [F * C <= g]
prob = cp.Problem(Objective, Constraints)
answer = prob.solve()
print(f'Cost for worst case in robust LP problem is {answer}')

# nominal LP problem
Objective = cp.Minimize(C_nom.T * X)
Constraints = [A * X >= B]
prob = cp.Problem(Objective, Constraints)
prob.solve()
print(f'Optimal cost from nominal LP problem is {float(C_nom.T * X.value)}')

C = cp.Variable((n, 1))
Objective = cp.Maximize(C.T * X.value)
Constraints = [F * C <= g]
prob = cp.Problem(Objective, Constraints)
answer = prob.solve()
print(f'Cost for worst case in nominal LP problem is {answer}')

Optimal cost from robust LP problem is 2.5386894180454997
Cost for worst case in robust LP problem is 2.8767939167888232
Optimal cost from nominal LP problem is 2.1092714620826003
Cost for worst case in nominal LP problem is 7.221562204280983
